<a href="https://colab.research.google.com/github/sairagillani18k/NLP/blob/main/Sentiment_analysis_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf

In [3]:
num_gpus_available = len(tf.config.experimental.list_physical_devices('GPU'))
print("Num GPUs Available: " , num_gpus_available)
assert num_gpus_available >0

Num GPUs Available:  1


In [ ]:
!pip install transformers

In [6]:
from transformers import DistilBertTokenizerFast
from transformers import TFDistilBertForSequenceClassification
import pandas as pd
import numpy as np

In [9]:
import nltk
import re
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
import tensorflow_datasets as tfds
ds = tfds.load('amazon_us_reviews/Mobile_Electronics_v1_00' , split = 'train' , shuffle_files = True)
assert isinstance(ds, tf.data.Dataset)
print(ds)

In [14]:
df = tfds.as_dataframe(ds)

In [15]:
df.head()

,data/customer_id,data/helpful_votes,data/marketplace,data/product_category,data/product_id,data/product_parent,data/product_title,data/review_body,data/review_date,data/review_headline,data/review_id,data/star_rating,data/total_votes,data/verified_purchase,data/vine
0,b'20980074',0,b'US',b'Mobile_Electronics',b'B00D1847NE',b'274617424',b'Teenage Mutant Ninja Turtles Boombox CD Play...,b'Does not work',b'2015-01-09',b'One Star',b'R1OVS0D6SEXPW7',1,0,0,1
1,b'779273',0,b'US',b'Mobile_Electronics',b'B00KMO6DYG',b'397452138',b'4 Gauge Amp Kit Amplifier Install Wiring Com...,b'This is a great wiring kit i used it to set ...,b'2015-08-06',b'Great kit',b'R9VSD0ET8FERB',4,0,0,1
2,b'15410531',0,b'US',b'Mobile_Electronics',b'B000GWLL0K',b'948304826',b'Travel Wall Charger fits Creative Zen Vision...,b'It works great so much faster than USB charg...,b'2007-03-15',b'A/C Charger for Creative Zen Vision M',b'R3ISXCZHWLJLBH',5,0,0,1
3,b'27389005',0,b'US',b'Mobile_Electronics',b'B008L3JE6Y',b'466340015',b'High Grade Robust 360\xc2\xb0 Adjustable Car...,b'This product was purchased to hold a monitor...,b'2013-07-30',b'camera stand',b'R1TWVUDOFJSQAW',5,0,0,1
4,b'2663569',0,b'US',b'Mobile_Electronics',b'B00GHZS4SC',b'350592810',b'HDE Multifunctional Bluetooth FM Audio Car K...,"b""it works but it has really bad sound quality...",b'2014-12-31',b'bad sound quality',b'R2PEOEUR1LP0GH',3,0,0,1


In [16]:
df['Sentiment'] = df["data/star_rating"].apply(lambda score: "positive" if score >=3 else "negative")
df['Sentiment'] = df['Sentiment'].map({'positive':1 , 'negative': 0})
df['short_review']= df['data/review_body'].str.decode("utf-8")
df= df[["short_review", "Sentiment"]]

In [17]:
df.head()

,short_review,Sentiment
0,Does not work,0
1,This is a great wiring kit i used it to set up...,1
2,It works great so much faster than USB charger...,1
3,This product was purchased to hold a monitor o...,1
4,it works but it has really bad sound quality. ...,1


In [18]:
n= 54975
df.drop(df.tail(n).index, inplace = True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [19]:
reviews = df['short_review'].values.tolist()
labels = df['Sentiment'].tolist()

In [21]:
from sklearn.model_selection import train_test_split
training_sentences , validation_sentences, training_labels, validation_labels= train_test_split(reviews , labels, test_size = 0.2)

In [22]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [24]:
tokenizer([training_sentences[0]], truncation = True , padding = True , max_length = 128)

{'input_ids': [[101, 2573, 2307, 1026, 7987, 1013, 1028, 2053, 3471, 1998, 1996, 16500, 2001, 3733, 1026, 7987, 1013, 1028, 1996, 3976, 2001, 2157, 1026, 7987, 1013, 1028, 1045, 2052, 16755, 1996, 13438, 2000, 3087, 1026, 7987, 1013, 1028, 2043, 6419, 1996, 13438, 102]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

In [25]:
train_encodings = tokenizer(training_sentences , truncation = True , padding = True)
val_encodings = tokenizer(validation_sentences , truncation = True , padding = True)

In [26]:
train_dataset = tf.data.Dataset.from_tensor_slices((dict(train_encodings), training_labels))

In [27]:
val_dataset = tf.data.Dataset.from_tensor_slices((dict(val_encodings), validation_labels))

In [29]:
model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels= 2)

Downloading:   0%|          | 0.00/347M [00:00<?, ?B/s]

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_layer_norm', 'activation_13', 'vocab_projector', 'vocab_transform']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['dropout_19', 'classifier', 'pre_classifier']
You should probably TRAIN this model on a down-stream task to be able to use i

In [32]:
from tensorflow.python.ops.gen_dataset_ops import optimize_dataset
optimizer = tf.keras.optimizers.Adam(learning_rate = 5e-5 , epsilon = 1e-8)
model.compile(optimizer = optimizer , loss = model.compute_loss , metrics = ['accuracy'])

In [48]:
test_sentence = "This is a good product. Love it"

In [49]:
predict_input = tokenizer.encode(test_sentence, truncation = True, padding = True , return_tensors = "tf")

In [56]:
!pip install termcolor

In [60]:
tf_output = model.predict(predict_input)[0]
tf_prediction = tf.nn.softmax(tf_output, axis = 1)
labels = ['Negative  :(' , 'Positive   :)']
label = tf.argmax(tf_prediction, axis=1)
label = label.numpy()
print((labels[label[0]]))

Positive   :)
